In [1]:
import csv
import glob
import os
import random

from datasets import load_dataset, Dataset, DatasetDict

In [2]:
dataset = load_dataset("joetey/functions_annotated_with_intents")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2768 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /home/diwank/.cache/huggingface/datasets/joetey___parquet/joetey--functions_annotated_with_intents-d843a29a4c86a8ce/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset["train"][0]

{'function': 'def bfs(graph, s, t, parent):\n    # Return True if there is node that has not iterated.\n    visited = [False] * len(graph)\n    queue = [s]\n    visited[s] = True\n\n    while queue:\n        u = queue.pop(0)\n        for ind in range(len(graph[u])):\n            if visited[ind] is False and graph[u][ind] > 0:\n                queue.append(ind)\n                visited[ind] = True\n                parent[ind] = u\n\n    return visited[t]',
 'intent_category': 'networking_flow',
 'purpose': 'to find the shortest path from source to sink',
 'code_trans': ' breadth - first search of a graph .',
 'detailed_description': 'This function performs a breadth-first search on a graph. It takes in a graph, a starting node (s), an ending node (t), and a parent array. It then uses a queue to traverse the graph, marking each node'}

In [4]:
make_chatml = lambda name, role, content: dict(
    name=name, role=role, content=content,
)

system = lambda name, content: make_chatml(
    role="system",
    name=name,
    content=content,
)

situation = lambda content: system(name="situation", content=content)
thought = lambda content: system(name="thought", content=content)
information = lambda content: system(name="information", content=content)
me = lambda content, name=None: make_chatml(
    role="assistant",
    content=content,
    name=name,
)

person = lambda content, name=None: make_chatml(
    role="user",
    content=content,
    name=name,
)

def to_chatml(row):
    function = row["function"]
    intent_category = row["intent_category"]
    purpose = row["purpose"]
    code_trans = row["code_trans"]
    detailed_description = row["detailed_description"]

    chatml = [
        situation(
            f"Person is discussing a function written in Python with Samantha."
        ),
        person(
            f"Hey Samantha! Can you explain this Python function to me?\n\n```python\n{function}```"
        ),
        thought(
            f'This is a {intent_category} function. Its purpose is {purpose.strip() or code_trans}.'
        ),
        me(detailed_description, name="Samantha"),
    ]

    return dict(chatml=chatml)

In [7]:
dataset = dataset.map(to_chatml).remove_columns(list(set(dataset["train"].column_names) - {"chatml"}))

Loading cached processed dataset at /home/diwank/.cache/huggingface/datasets/joetey___parquet/joetey--functions_annotated_with_intents-d843a29a4c86a8ce/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-71e721a650f1a51d.arrow


In [8]:
dataset.push_to_hub("diwank/samantha-function-explanation", private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]